In [ ]:
import pandas as pd
import numpy as np

from google.colab import files
import ipywidgets as widgets
from google.colab import drive

# Uploading files & Input

In [ ]:
origin_file = files.upload()
origin_df = pd.read_excel(list(origin_file.keys())[0])
origin_column_picker = widgets.Dropdown(options=list(origin_df.columns),description="Select Origin URLs Column:", value=None)
origin_column_picker

Saving origin-URLs.xlsx to origin-URLs (2).xlsx


Dropdown(description='Select Origin URLs Column:', options=('Address', 'Content Type', 'Status Code', 'Status'…

In [ ]:
destination_file = files.upload()
destination_df = pd.read_excel(list(destination_file.keys())[0])
dest_column_picker = widgets.Dropdown(options=list(destination_df.columns),description="Select Destination URLs Column:",value=None)
dest_column_picker

Saving destination-URLs.xlsx to destination-URLs (2).xlsx


Dropdown(description='Select Destination URLs Column:', options=('Address', 'Content Type', 'Status Code', 'St…

In [ ]:
origin = origin_df[['Address',origin_column_picker.value]]
dest = destination_df[['Address',dest_column_picker.value]]

# data cleaning





removing special characters

In [ ]:
import string

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

origin_cleaned = origin[origin_column_picker.value].apply(lambda text: remove_punctuation(text))
dest_cleaned = dest[dest_column_picker.value].apply(lambda text: remove_punctuation(text))

removing extra spaces

In [ ]:
import re

origin_space_removed = []
dest_space_removed = []

for i in origin_cleaned:
  temp = re.sub(' +', ' ', i)
  origin_space_removed.append(temp)

for i in dest_cleaned:
  temp = re.sub(' +', ' ', i) # replacing multiple consecutive spaces with a single space
  dest_space_removed.append(temp)

converting all to small case for better matching results

In [ ]:
origin_small_cased = []
dest_small_cased = []

for i in origin_cleaned:
  temp = i.lower()
  origin_small_cased.append(temp)

for i in dest_cleaned:
  temp = i.lower() 
  dest_small_cased.append(temp)

# Vectorization

joining 2 lists 

then vectorize

then split back

this way we'll be able to create origin & destination vectors with same dimensions, which is required in cosine similarity

In [ ]:
len_origin = len(origin_small_cased)

temp_list = origin_small_cased + dest_small_cased
#len(temp_list)

Bag of words

In [ ]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(temp_list).toarray()
#X.shape

In [ ]:
origin_vectorized = X[0:len(origin_small_cased)]
dest_vectorized = X[len(origin_small_cased):len(temp_list)]
#origin_vectorized.shape

In [ ]:
origin_with_vect = pd.DataFrame()

#origin_with_vect['origin'] = origin_small_cased
origin_with_vect['origin_vectors'] = list(origin_vectorized)
origin_with_vect['from'] = origin_df[origin_column_picker.value]
origin_with_vect['from address'] = origin_df['Address']

dest_with_vect = pd.DataFrame()

#dest_with_vect['dest'] = dest_small_cased
dest_with_vect['dest_vectors'] = list(dest_vectorized)
dest_with_vect['to'] =  destination_df[dest_column_picker.value]
dest_with_vect['to address'] = destination_df['Address']

#dest_with_vect.head(2)

# Matching using cosine similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(list(origin_with_vect['origin_vectors']),list(dest_with_vect['dest_vectors']))
from_ = []
from_addr = []

to = []
to_addr = []

simil = []

for i in origin_with_vect.index:
  # step 1, enter the origin val
  origin_col = origin_with_vect['from'].loc[i]
  origin_add = origin_with_vect['from address'].loc[i]

  list_of_similarities = sorted(list(enumerate(similarity[i])),reverse=True,key = lambda x: x[1])

  dest_index = list_of_similarities[0][0]
  similarity_val = list_of_similarities[0][1]

  des_col = dest_with_vect['to'].loc[dest_index]
  des_add = dest_with_vect['to address'].loc[dest_index]


  from_.append(origin_col)
  from_addr.append(origin_add)

  to.append(des_col)
  to_addr.append(des_add)

  simil.append(similarity_val)

#temp = pd.DataFrame(columns = ['from' , 'from address' , 'to' , 'to address' , 'similarity'])
temp = pd.DataFrame({'from': from_, 'from address': from_addr, 'to':to , 'to address':to_addr , 'similarity':simil})

temp = temp.sort_values(by='similarity' , ascending=False)

temp

,from,from address,to,to address,similarity
310,Basix Tackle Box | Basix | Fishing Tackle | Korda,http://www.korda.co.uk/fishingtackle/basix/bas...,Basix Tackle Box | Korda,https://korda.withcandour.co.uk/products/basix...,0.904534
0,Dark Matter Braid | Dark Matter | Fishing Tack...,http://www.korda.co.uk/fishingtackle/dark-matt...,Dark Matter Braid | Korda,https://korda.withcandour.co.uk/products/dark-...,0.866025
4,Dark Matter Putty | Dark Matter | Fishing Tack...,http://www.korda.co.uk/fishingtackle/dark-matt...,Dark Matter Putty | Korda,https://korda.withcandour.co.uk/products/dark-...,0.866025
306,Basix Hair Rigs Wide Gape | Basix | Fishing Ta...,http://www.korda.co.uk/fishingtackle/basix/bas...,Basix Hair Rigs Wide Gape | Korda,https://korda.withcandour.co.uk/products/basix...,0.861640
307,Basix Lead Clip Action Pack | Basix | Fishing ...,http://www.korda.co.uk/fishingtackle/basix/bas...,Basix Lead Clip Action Pack | Korda,https://korda.withcandour.co.uk/products/basix...,0.861640
...,...,...,...,...,...
197,Ready-Tied IQ D-Rig | Line and Hook Links | Fi...,http://www.korda.co.uk/fishingtackle/line-and-...,Carp Line | Korda,https://korda.withcandour.co.uk/products/carp-...,0.365148
214,Ready Tied Hinge Rigs | Line and Hook Links | ...,http://www.korda.co.uk/fishingtackle/line-and-...,Carp Line | Korda,https://korda.withcandour.co.uk/products/carp-...,0.348155
92,Ali Hamidi's Carp Fishing Masterclass | DVD/Bo...,http://www.korda.co.uk/fishingtackle/dvds-book...,Carp Line | Korda,https://korda.withcandour.co.uk/products/carp-...,0.348155
90,The Big Fish Off Guide To Fishing | DVD/Books ...,http://www.korda.co.uk/fishingtackle/dvds-book...,Big Grippa | Korda,https://korda.withcandour.co.uk/products/big-g...,0.320256


In [ ]:
result_file = temp.to_excel("match_results.xlsx")
files.download("match_results.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>